<a href="https://colab.research.google.com/github/DeepsMoseli/TswanaBert/blob/master/tswanaBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 757kB 2.8MB/s 
     |████████████████████████████████| 1.1MB 13.9MB/s 
     |████████████████████████████████| 3.0MB 19.1MB/s 
     |████████████████████████████████| 890kB 26.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=2f548d36083c59eda087998f662536a142446fb585f9a5bf5e4b396e457df146
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [14]:
from pathlib import Path

from tokenizers import ByteLevelBPETokenizer

paths = [str(x) for x in Path("./").glob("**/*.txt")]
paths

['large_test_clean.txt']

## initialize tokenizer and customize training


In [20]:
tokenizer = ByteLevelBPETokenizer()

# Customize training
tokenizer.train(files=["./large_test_clean.txt"], show_progress=True,vocab_size=25000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])
tokenizer.save_model('.','tswanaBert')
print("Trained vocab size: {}".format(tokenizer.get_vocab_size()))

Trained vocab size: 13333


In [30]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing


tokenizer = ByteLevelBPETokenizer(
    "./TswanaBert/vocab.json",
    "./TswanaBert/merges.txt",
)

tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)


encoding = tokenizer.encode("gore fa a re merafe yotlhe e nne le.")
print("Encoded String: ", encoding.tokens)

decoded = tokenizer.decode(encoding.ids)
print("Decoded string: {}".format(decoded))

Encoded String:  ['<s>', 'gore', 'Ġfa', 'Ġa', 'Ġre', 'Ġmerafe', 'Ġyotlhe', 'Ġe', 'Ġnne', 'Ġle', '.', '</s>']
Decoded string: <s>gore fa a re merafe yotlhe e nne le.</s>


## Make and Train a language model



In [24]:
!nvidia-smi

Tue Jun 30 10:30:31 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    30W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [25]:
import torch
torch.cuda.is_available()

True

### Config the model and re-create our tokenizer in transformers


In [31]:
from transformers import RobertaConfig
from transformers import RobertaTokenizerFast

config = RobertaConfig(
    vocab_size=13333,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

tokenizer = RobertaTokenizerFast.from_pretrained("./TswanaBert", max_len=512)

 since we are training from scratch, we initialize from config not a checkpoint of saved model


In [34]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config = config)
print("Number of parameters: ", model.num_parameters())

Number of parameters:  54360085


## Make and register training dataset

In [36]:
%%time
from transformers import LineByLineTextDataset
from transformers import DataCollatorForLanguageModeling

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="./large_test_clean.txt",
    block_size=128,
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

CPU times: user 905 ms, sys: 43.2 ms, total: 948 ms
Wall time: 559 ms


Initialize model

In [39]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./TswanaBert",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_gpu_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    prediction_loss_only=True,
)

In [ ]:
%%time
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


In [ ]:
trainer.save_model("./TswanaBert")

-----
# Now lets Test

In [ ]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./TswanaBert",
    tokenizer="./TswanaBert"
)

fill_mask("Batho botlhe ba tsetswe ba <mask>.")